In [3]:
!pip3 install nb_black
!pip3 install flake8 pycodestyle_magic
!pip3 install bigquery
!pip3 install db-dtypes

!pip3 install matplotlib
!pip3 install seaborn
!pip3 install tqdm

  Using cached nb_black-1.0.7-py3-none-any.whl
  Using cached pycodestyle_magic-0.5-py2.py3-none-any.whl (9.5 kB)
  Using cached bigquery-0.0.34.tar.gz (8.1 kB)
  Preparing metadata (setup.py) ... done
  Using cached dbstream-0.1.19-py3-none-any.whl
  Using cached google_cloud_bigquery-3.2.0-py2.py3-none-any.whl (211 kB)
  Using cached googleauthentication-0.0.17-py3-none-any.whl
  Using cached google_cloud_bigquery_storage-2.13.2-py2.py3-none-any.whl (180 kB)
  Using cached bigquery-0.0.33.tar.gz (7.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached bigquery-0.0.32.tar.gz (7.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached bigquery-0.0.31.tar.gz (2.0 kB)
  Preparing metadata (setup.py) ... done
  Using cached bigquery-0.0.30-py3-none-any.whl
  Using cached google_api_core-2.8.2-py3-none-any.whl (114 kB)
  Using cached dacktool-0.0.7-py3-none-any.whl
  Using cached proto_plus-1.22.1-py3-none-any.whl (47 kB)
  Using cached google_cloud_core-2.3.1-py2.py3-none-any

In [1]:
from google.cloud import bigquery
from datetime import datetime, timedelta

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
from datetime import datetime
import datetime
from tqdm import tqdm
from matplotlib.patches import Rectangle
import warnings

warnings.filterwarnings("ignore")

%load_ext google.cloud.bigquery
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from scipy.special import comb
from itertools import combinations, permutations


<IPython.core.display.Javascript object>

Referência: https://www.computersciencemaster.com.br/como-funciona-o-algoritmo-apriori/

In [3]:
df_mining = pd.DataFrame(
    [
        [
            "onion",
            "potato",
            "burguer",
            "false",
            "false"
        ],
        [
            "false",
            "potato",
            "burguer",
            "milk",
            "false"
        ],
        ["false", "false", "false", "milk", "beer"],
        ["onion", "potato", "false", "milk", "false"],
        ["onion", "potato", "burguer", "false", "beer"],
        ["onion", "potato", "burguer", "milk", "beer"],
    ]
)

<IPython.core.display.Javascript object>

In [4]:
df_mining

,0,1,2,3,4
0,onion,potato,burguer,false,false
1,false,potato,burguer,milk,false
2,false,false,false,milk,beer
3,onion,potato,false,milk,false
4,onion,potato,burguer,false,beer
5,onion,potato,burguer,milk,beer


<IPython.core.display.Javascript object>

In [5]:
minimum_support = 0.5
df_values = df_mining.values.astype(str)
n_trans = len(df_mining)
n_minimum_support = minimum_support * n_trans

index, counts = np.unique(df_values, return_counts=True)
df_item = pd.DataFrame(zip(index, counts), columns=["produto", "frequencia"])
df_item.sort_values(by="frequencia", ascending=False, inplace=True)
df_item.reset_index(drop=True, inplace=True)
df_item = df_item.query("produto != 'false'")
df_item

,produto,frequencia
1,potato,5
2,burguer,4
3,milk,4
4,onion,4
5,beer,3


<IPython.core.display.Javascript object>

In [6]:
n_minimum_support = minimum_support * n_trans
df_item_frequent = df_item[df_item["frequencia"] > n_minimum_support]
df_item_frequent["supp"] = df_item_frequent["frequencia"] / n_trans
df_item_frequent

,produto,frequencia,supp
1,potato,5,0.833333
2,burguer,4,0.666667
3,milk,4,0.666667
4,onion,4,0.666667


<IPython.core.display.Javascript object>

In [7]:
itemset_frequencia = []  # pd.DataFrame(columns=["itemset", "frequencia"])
for i in tqdm(np.arange(len(df_item_frequent), 0, -1)):
    comb = list(combinations(df_item_frequent["produto"].values, i))
    for w in comb:
        count = 0
        for instancia in df_values:
            if all(elem in instancia for elem in w):
                count = count + 1
        if count >= n_minimum_support:
            itemset_frequencia.append({"itemset": w, "frequencia": count})
df_itemset_frequencia = pd.DataFrame(itemset_frequencia)
df_itemset_frequencia.sort_values(by="frequencia", inplace=True, ascending=False)
df_item_frequent.set_index("produto", inplace=True)
df_itemset_frequencia["supp"] = df_itemset_frequencia["frequencia"] / n_trans
df_itemset_frequencia

100%|██████████| 4/4 [00:00<00:00, 3827.79it/s]


,itemset,frequencia,supp
5,"(potato,)",5,0.833333
1,"(potato, burguer)",4,0.666667
3,"(potato, onion)",4,0.666667
6,"(burguer,)",4,0.666667
7,"(milk,)",4,0.666667
8,"(onion,)",4,0.666667
0,"(potato, burguer, onion)",3,0.500000
2,"(potato, milk)",3,0.500000
4,"(burguer, onion)",3,0.500000


<IPython.core.display.Javascript object>

In [8]:
df_itemset_frequencia

,itemset,frequencia,supp
5,"(potato,)",5,0.833333
1,"(potato, burguer)",4,0.666667
3,"(potato, onion)",4,0.666667
6,"(burguer,)",4,0.666667
7,"(milk,)",4,0.666667
8,"(onion,)",4,0.666667
0,"(potato, burguer, onion)",3,0.500000
2,"(potato, milk)",3,0.500000
4,"(burguer, onion)",3,0.500000


<IPython.core.display.Javascript object>

# Support

A probabilidade da combinação x acontecer em relação ao total de operações 

In [9]:
df_itemset_frequencia["supp"] = df_itemset_frequencia["frequencia"] / n_trans

<IPython.core.display.Javascript object>

In [10]:
df_itemset_frequencia

,itemset,frequencia,supp
5,"(potato,)",5,0.833333
1,"(potato, burguer)",4,0.666667
3,"(potato, onion)",4,0.666667
6,"(burguer,)",4,0.666667
7,"(milk,)",4,0.666667
8,"(onion,)",4,0.666667
0,"(potato, burguer, onion)",3,0.500000
2,"(potato, milk)",3,0.500000
4,"(burguer, onion)",3,0.500000


<IPython.core.display.Javascript object>

In [11]:
df_item_frequent

,frequencia,supp
produto,,
potato,5,0.833333
burguer,4,0.666667
milk,4,0.666667
onion,4,0.666667


<IPython.core.display.Javascript object>

In [12]:
prob = []
for item, supp in tqdm(df_itemset_frequencia[["itemset", "supp"]].values):
    s = round(supp * 100, 2)
    for conjunto, supp_c in df_itemset_frequencia[["itemset", "supp"]].values:

        conf = round((supp_c / supp) * 100, 2)
        lift = round(
            supp_c / np.prod([df_item_frequent.loc[i]["supp"] for i in conjunto]), 2
        )
        conj = set(conjunto) - set(item)
        if set(item).intersection(set(conjunto)) == set(item):
            if len(conj) > 0 and conf > 0:
                prob.append([conj, item, conf, lift])
df = pd.DataFrame(prob, columns=["B", "A", "confidence", "lift"])
df

100%|██████████| 9/9 [00:00<00:00, 335.77it/s]


,B,A,confidence,lift
0,{burguer},"(potato,)",80.0,1.20
1,{onion},"(potato,)",80.0,1.20
2,"{onion, burguer}","(potato,)",60.0,1.35
3,{milk},"(potato,)",60.0,0.90
4,{onion},"(potato, burguer)",75.0,1.35
5,{burguer},"(potato, onion)",75.0,1.35
6,{potato},"(burguer,)",100.0,1.20
7,"{potato, onion}","(burguer,)",75.0,1.35
8,{onion},"(burguer,)",75.0,1.12
9,{potato},"(milk,)",75.0,0.90


<IPython.core.display.Javascript object>

In [13]:
df_mining[(df_mining[0] == "onion")]

,0,1,2,3,4
0,onion,potato,burguer,false,false
3,onion,potato,false,milk,false
4,onion,potato,burguer,false,beer
5,onion,potato,burguer,milk,beer


<IPython.core.display.Javascript object>

In [14]:
df_mining[(df_mining[1] == "potato")]

,0,1,2,3,4
0,onion,potato,burguer,false,false
1,false,potato,burguer,milk,false
3,onion,potato,false,milk,false
4,onion,potato,burguer,false,beer
5,onion,potato,burguer,milk,beer


<IPython.core.display.Javascript object>

# Confidence 

A probabilidade condicional da combinação dada outra combinação

probabilidade de comprar burguer dado que comprou potato 

In [15]:
0.66666666 / 0.83333333333

0.7999999920031999

<IPython.core.display.Javascript object>

ou seja 80% das transações que potato, também compram burguer. 

5 transações de potato, dessas 4 compram burguer

In [16]:
df_mining[(df_mining[1] == "potato")]

,0,1,2,3,4
0,onion,potato,burguer,false,false
1,false,potato,burguer,milk,false
3,onion,potato,false,milk,false
4,onion,potato,burguer,false,beer
5,onion,potato,burguer,milk,beer


<IPython.core.display.Javascript object>

outro exemplo:

probabilidade de comprar burguer and onion dado que comprou potato 

In [17]:
0.5 / 0.5

1.0

<IPython.core.display.Javascript object>

isso significa que em todas as vezes que se comprou burguer e onion também se comprou potato

In [18]:
df_mining[(df_mining[0] == "onion") & (df_mining[2] == "burguer")]

,0,1,2,3,4
0,onion,potato,burguer,false,false
4,onion,potato,burguer,false,beer
5,onion,potato,burguer,milk,beer


<IPython.core.display.Javascript object>

probabilidade de comprar potato, dado que comprou burguer and onion

In [19]:
0.5 / 0.83

0.6024096385542169

<IPython.core.display.Javascript object>

In [20]:
df_mining[(df_mining[1] == "potato")]

,0,1,2,3,4
0,onion,potato,burguer,false,false
1,false,potato,burguer,milk,false
3,onion,potato,false,milk,false
4,onion,potato,burguer,false,beer
5,onion,potato,burguer,milk,beer


<IPython.core.display.Javascript object>

In [21]:
3 / 5

0.6

<IPython.core.display.Javascript object>

# lift

Esta medida calcula também a possibilidade de um item ser comprado em relação a outro item. Porém, esta medida considera a popularidade de ambos os itens.

lift de comprar  burguer dado que comprou potato

In [22]:
0.66666666 / (0.833333 * 0.666667)

1.199999868000253

<IPython.core.display.Javascript object>